In [1]:
import math
import copy
import statistics
import numpy as np
import Parsing_midi as pm
import Init_chromo as init
import mido
import matplotlib.pyplot as plt
from mido import Message, MidiFile, MidiTrack
from collections import Counter

In [2]:
midi_list = pm.get_midi('MidiSample/sky.mid', 0)
note_msg, midi_msg = pm.separate_song(midi_list, 15360)
m = midi_msg[1]
n = midi_msg[2]

ori = pm.to_midi_file(m, 8 , 'ori.mid')

In [3]:
# 將note依照time拆成多個
def to_note_freq(midi_msg):     
    note_freq = []; count = 0
    for i in range(len(midi_msg)):
        if i % 2 == 0:
            for j in range(int((midi_msg[i].time + midi_msg[i+1].time)/240)):
                count += 1
                note_freq.append([midi_msg[i].note, count])
    return note_freq

# 計算面積差  
def count_area(ori_y, new_y):  
    area = 0
    for i in range(len(ori_y)):
        area += abs(ori_y[i]-new_y[i])
    return area

# smooth melody curve, sliding window = 7
def smooth(note_seq):  
    temp = []
    for i in range(len(note_seq)):
        if i < 4:
            temp.append(note_seq[i])
        elif i > len(note_seq) - 4:
            temp.append(note_seq[i])
        else:
            median = int(statistics.median(note_seq[i-4:i+4]))
            temp.append(median)
    return temp


def count_area_score(ori_midi, new_midi, smooth):  # smooth
    score = 0
    a = mm.to_note_freq(ori_midi)
    b = mm.to_note_freq(new_midi)
    a, b = mm.correct_xy(a,b)
    # 拆成XY座標
    x = [i[1] for i in a]
    y = [i[0] for i in a]

    x2 = [i[1] for i in b]
    y2 = [i[0] for i in b]
    
    print(len(y),len(y2))

    # smooth
    y_s1 = mm.smooth(y)
    y_s2 = mm.smooth(y2)

    if smooth is True:
        # count area
        area_diff = mm.count_area(y, y_s2)

        if area_diff > 0 and area_diff <= 20:
            score += 40
        elif area_diff > 20 and area_diff <= 30:
            score += 35
        elif area_diff > 30 and area_diff <= 40:
            score += 30
        elif area_diff > 40 and area_diff <= 50:
            score += 25
        elif area_diff > 50 and area_diff <= 60:
            score += 20
        elif area_diff > 60 and area_diff <= 70:
            score += 15
        elif area_diff > 70 and area_diff <= 80:
            score += 10
        elif area_diff > 80 and area_diff <= 90:
            score += 5
        else:
            score += 0 
    else:  
        area_diff = mm.count_area(y, y2)
        if area_diff > 0 and area_diff <= 20:
            score += 40
        elif area_diff > 20 and area_diff <= 30:
            score += 35
        elif area_diff > 30 and area_diff <= 40:
            score += 30
        elif area_diff > 40 and area_diff <= 50:
            score += 25
        elif area_diff > 50 and area_diff <= 60:
            score += 20
        elif area_diff > 60 and area_diff <= 70:
            score += 15
        elif area_diff > 70 and area_diff <= 80:
            score += 10
        elif area_diff > 80 and area_diff <= 90:
            score += 5
        else:
            score += 0 
    return score

In [6]:
# 校正頭尾
def correct_xy(xy1, xy2):
    if len(xy1) > len(xy2):
        xy1 = xy1[0:len(xy2)]
    else:
        xy2 = xy2[0:len(xy1)]
    return xy1, xy2